# Example Notebook For Using Optimizer Tools

In [1]:
import pandas as pd
import datetime as dt

from fps_dfs_optimizer.src.draftkings_tools import get_contests, get_players, EntriesHandler, get_today, get_players_from_salaries, get_full_slate
from fps_dfs_optimizer.src.projections import get_projections
from fps_dfs_optimizer.src.optimizer import Exposures, LineupOptimizer
from fps_dfs_optimizer.src.generator import LineupGenerator

## Generate today's players sheet
This block will retrieve the players DataFrame for the current date.
Any projections script can be implemented in get_projections(), but right now dummy projections are given

In [2]:
today = get_today()
# today = dt.date(2021, 1, 1)
df_contests = get_contests(today)

In [4]:
df_full, df_night = get_full_slate(df_contests)

NameError: name 'get_full_slate' is not defined

In [83]:
df_full

,id,double_up,draft_group_id,entries,fantasy_player_points,fifty_fifty,guaranteed,head_to_head,name,payout,sport,starred,starts_at,maximum,fee,total


In [69]:
def get_players(draft_group_id):
    draft_players = draftables(draft_group_id)
    df_players = pd.DataFrame(draft_players['draftables'])
    print(df_players)
    names = list(df_players['names'].values)
    df_names = pd.DataFrame(names)
    df_players = pd.concat((df_players, df_names),axis=1)
    competitions = list(df_players['competition'].values)
    df_comps = pd.DataFrame(competitions)
    df_comps.columns = ['contest id', 'contest names', 'start time']
    df_players = pd.concat((df_players, df_comps),axis=1)
    df_players.reset_index(0, drop=True, inplace=True)
    map_cols = {'display' : 'Name', 'id' : 'ID', 'position' : 'Position',
        'news_status' : 'Status', 'salary' : 'Salary', 
        'contest names' : 'Game', 'start time' : 'Time',
        'team_abbreviation' : 'TeamAbbrev'}
    df_players.rename(columns=map_cols, inplace=True)
    df_players['Time'] = pd.to_datetime(df_players['Time'])
    return df_players[list(map_cols.values())]

In [70]:
group_id = df_night.draft_group_id.unique()[0]
df_players = get_players(group_id)

           id  player_id position  roster_slot_id  salary  swappable  \
0    16049653     395388    PG/SG             458   11200       True   
1    16049654     395388    PG/SG             459   11200       True   
2    16049655     395388    PG/SG             464   11200       True   
3    16049656     395388    PG/SG             465   11200       True   
4    16049665     463121       PG             458   10300       True   
..        ...        ...      ...             ...     ...        ...   
220  16049834    1135177       SG             465    3000       True   
221  16049835    1138391    SF/PF             460    3000       True   
222  16049836    1138391    SF/PF             461    3000       True   
223  16049837    1138391    SF/PF             463    3000       True   
224  16049838    1138391    SF/PF             465    3000       True   

     disabled news_status  team_id team_abbreviation draft_alerts  \
0       False      Recent       10               HOU           [] 

In [64]:
df_players

,Name,ID,Position,Status,Salary,Game,Time,TeamAbbrev
0,James Harden,16049653,PG/SG,Recent,11200,HOU @ DEN,2020-12-29 02:00:00+00:00,HOU
1,James Harden,16049654,PG/SG,Recent,11200,HOU @ DEN,2020-12-29 02:00:00+00:00,HOU
2,James Harden,16049655,PG/SG,Recent,11200,HOU @ DEN,2020-12-29 02:00:00+00:00,HOU
3,James Harden,16049656,PG/SG,Recent,11200,HOU @ DEN,2020-12-29 02:00:00+00:00,HOU
4,Damian Lillard,16049665,PG,Recent,10300,POR @ LAL,2020-12-29 03:00:00+00:00,POR
...,...,...,...,...,...,...,...,...
220,CJ Elleby,16049834,SG,None,3000,POR @ LAL,2020-12-29 03:00:00+00:00,POR
221,Nassir Little,16049835,SF/PF,Recent,3000,POR @ LAL,2020-12-29 03:00:00+00:00,POR
222,Nassir Little,16049836,SF/PF,Recent,3000,POR @ LAL,2020-12-29 03:00:00+00:00,POR
223,Nassir Little,16049837,SF/PF,Recent,3000,POR @ LAL,2020-12-29 03:00:00+00:00,POR


## Optionally, can read players sheet directly from DKSalaries, if it is downloaded

In [5]:
# df_players = get_players_from_salaries('../data/raw/DKSalaries_test.csv')

In [6]:
points, std = get_projections(df_players)
df_players['projections'] = points
df_players['std'] = std

In [7]:
df_players.head()

,Name,ID,Position,Status,Salary,Game,Time,TeamAbbrev,projections,std
0,LeBron James,15580981,PG/SF,Recent,18600,MIA @ LAL,2020-10-10 01:00:00+00:00,LAL,91.364916,25.382058
1,Anthony Davis,15580982,PF/C,Recent,17700,MIA @ LAL,2020-10-10 01:00:00+00:00,LAL,113.327819,48.794305
2,Jimmy Butler,15580983,SG/SF,Recent,16200,MIA @ LAL,2020-10-10 01:00:00+00:00,MIA,62.325117,26.971521
3,Bam Adebayo,15580984,C,Recent,14700,MIA @ LAL,2020-10-10 01:00:00+00:00,MIA,62.372825,23.083821
4,Tyler Herro,15580985,SG,Breaking,13500,MIA @ LAL,2020-10-10 01:00:00+00:00,MIA,72.774675,22.805194


## Now exposures must be inputted
Here the dataframe above is saved to the data/temp folder for exposures to be filled out.
Once the exposures are filled out, run exp.read_exposures() to pull the sheet into the dataframe.

In [15]:
exp = Exposures(df_players)

Please fill out exposures in ../data/temp/20201006.csv


In [16]:
df_players = exp.read_exposures()

In [17]:
cols = df_players.columns
df_players

,Name,ID,Position,Salary,Status,TeamAbbrev,Time,projections,std,min_exp,max_exp
0,LeBron James,15580981,PG/SF,18600,Recent,LAL,2020-10-10 01:00:00+00:00,91.409624,46.703193,0,0.4
1,Anthony Davis,15580982,PF/C,17700,Recent,LAL,2020-10-10 01:00:00+00:00,89.181931,28.108531,0,0.4
2,Jimmy Butler,15580983,SG/SF,16200,Recent,MIA,2020-10-10 01:00:00+00:00,64.552638,32.301815,0,0.4
3,Bam Adebayo,15580984,C,14700,Recent,MIA,2020-10-10 01:00:00+00:00,57.276068,35.406902,0,0.4
4,Tyler Herro,15580985,SG,13500,Breaking,MIA,2020-10-10 01:00:00+00:00,69.263188,26.463128,0,0.4
...,...,...,...,...,...,...,...,...,...,...,...
65,Dion Waiters,15580976,SG/SF,1000,Breaking,LAL,2020-10-10 01:00:00+00:00,1.970700,2.253133,0,0.4
66,Quinn Cook,15580977,PG,1000,None,LAL,2020-10-10 01:00:00+00:00,5.666160,2.490163,0,0.4
67,Kyle Alexander,15580978,C,1000,None,MIA,2020-10-10 01:00:00+00:00,3.870371,2.635554,0,0.4
68,Avery Bradley,15580979,PG,1000,None,LAL,2020-10-10 01:00:00+00:00,3.638591,1.676683,0,0.4


# Now let's work with a historical dataframe 
Data is from 2020/01/02

In [18]:
df_players = pd.read_csv('../data/raw/test_20200102.csv')
df_players['Time'] = pd.to_datetime(df_players['Time'])
df_players = df_players[cols]
df_players.head()

,Name,ID,Position,Salary,Status,TeamAbbrev,Time,projections,std,min_exp,max_exp
0,Gary Harris,14066354,SG,4600,recent,MIL,2020-09-10 23:00:00+00:00,29.62,4.462711,0,0.4
1,Darius Garland,14066387,PG,4400,recent,HOU,2020-09-10 23:00:00+00:00,26.53,4.750068,0,0.4
2,Collin Sexton,14066177,PG/SG,5900,recent,HOU,2020-09-10 23:00:00+00:00,34.33,7.762624,0,0.4
3,Kevin Porter,14066428,SG/SF,4200,recent,MIL,2020-09-10 23:00:00+00:00,23.80,5.999999,0,0.4
4,Aaron Holiday,14066210,PG/SG,5700,recent,HOU,2020-09-10 23:00:00+00:00,31.38,5.332358,0,0.4


# Let's do a simple lineup optimization
This is like the original optimizer. Provide the number of desired lineups and the dataframe.
Note that order=1 is an optional argument, but defaults to unordered (0)
time_limit=10.0 by default, but is also an optional argument.

This first step creates the optimization model.

In [19]:
opt = LineupOptimizer(df_players, n_lineups=20, verbose=True)

## opt.solve() solves the optimization problem

In [21]:
opt.solve()

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               10

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

  40026 25539     5515.3287    41     5505.4500     5515.4356   300963    0.18%
Elapsed time = 0.13 sec. (8.21 ticks, tree = 17.88 MB, solutions = 14)
* 40740+25617                         5506.7900     5515.4356             0.16%
* 40801+25629                         5507.1200     5515.4356             0.15%
* 40829+25634                         5507.1800     5515.4356             0.15%
* 41371+25608                         5507.2600     5515.4356             0.15%
* 41415+25606                         5507.7600     5515.4356             0.14%
* 41711+21383                         5507.9900     5515.4356             0.14%
  41805 20964     5513.4899    41     5507.9900     5515.4356   30671

array([[0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1.,
        1., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0.,
        0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0.,
        1., 1., 0., 0.],
       [0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0.,
        1., 0., 1., 0.],
       [1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
        1., 1., 0., 0.],
       [0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0.,
        0., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0.,
        0., 1., 1., 1.],
       [0., 0., 1., 0., 1., 1.

## Now let's sort the optimal lineups

In [22]:
opt.sort_lineups()

2 Infeasible lineups dropped


,PG,SG,SF,PF,C,G,F,UTIL
0,Spencer Dinwiddie,Gary Harris,Kevin Porter,Robert Covington,Marquese Chriss,Aaron Holiday,Domantas Sabonis,Zach LaVine
1,De'Aaron Fox,Jalen Brunson,Dillon Brooks,Harrison Barnes,Richaun Holmes,Donovan Mitchell,Buddy Hield,Zach LaVine
2,Darius Garland,Jalen Brunson,Buddy Hield,Marquese Chriss,Domantas Sabonis,Aaron Holiday,Jimmy Butler,DeMar DeRozan
3,Jalen Brunson,Spencer Dinwiddie,Kevin Porter,LaMarcus Aldridge,Domantas Sabonis,Aaron Holiday,Dillon Brooks,Zach LaVine
4,Darius Garland,Collin Sexton,Dillon Brooks,Bojan Bogdanovic,Richaun Holmes,De'Aaron Fox,Buddy Hield,Zach LaVine
5,Jalen Brunson,Gary Harris,Kevin Porter,Bojan Bogdanovic,LaMarcus Aldridge,Spencer Dinwiddie,Domantas Sabonis,Shai Gilgeous-Alexander
6,Jalen Brunson,Spencer Dinwiddie,Kevin Porter,Domantas Sabonis,Ivica Zubac,Dillon Brooks,Jimmy Butler,Donovan Mitchell
7,Darius Garland,Collin Sexton,Buddy Hield,Bojan Bogdanovic,Marquese Chriss,Aaron Holiday,LaMarcus Aldridge,Donovan Mitchell
8,Collin Sexton,Gary Harris,Jimmy Butler,Bojan Bogdanovic,Ivica Zubac,De'Aaron Fox,Robert Covington,DeMar DeRozan
9,Darius Garland,Spencer Dinwiddie,Dillon Brooks,Bojan Bogdanovic,Ivica Zubac,Collin Sexton,Jimmy Butler,LaMarcus Aldridge


# Lineup Generator and Optimizer
Now we'll use a more complex lineup generation --> monte carlo simulation then select the lineups that have the highest likelihood of being a high scoring team.

<br> The default values are:
<br> var_multiple = 0.5 (we weight lineup projected variance at half the importance of the projected mean)
<br> drop_fraction = 0.5 (half the players are discarded each iteration of generation)
<br> time_limit = 1 (1 second is allowed for each optimization iteration)
<br> duplicates_lim = 100 (once 100 duplicate lineups have been generated, it kills the generator)

## First, let's demo the fast optimizer

In [24]:
fast_opt = LineupGenerator(df_players, n_lineups_to_optimize=150, batch_size=20, drop_fraction=0.05, time_limit=5, verbose=False)
fast_opt.generate(fast_opt.n_lineups_to_optimize)

1 Infeasible lineups dropped
19 Lineups
0 Duplicates
0 Infeasible lineups dropped
35 Lineups
0 Duplicates
1 Infeasible lineups dropped
53 Lineups
1 Duplicates
4 Infeasible lineups dropped
69 Lineups
1 Duplicates
1 Infeasible lineups dropped
74 Lineups
15 Duplicates
3 Infeasible lineups dropped
90 Lineups
16 Duplicates
1 Infeasible lineups dropped
105 Lineups
19 Duplicates
0 Infeasible lineups dropped
125 Lineups
19 Duplicates
4 Infeasible lineups dropped
141 Lineups
19 Duplicates
2 Infeasible lineups dropped
145 Lineups
22 Duplicates
0 Infeasible lineups dropped
149 Lineups
23 Duplicates
0 Infeasible lineups dropped
149 Lineups
24 Duplicates
0 Infeasible lineups dropped
149 Lineups
25 Duplicates
0 Infeasible lineups dropped
149 Lineups
26 Duplicates
0 Infeasible lineups dropped
149 Lineups
27 Duplicates
0 Infeasible lineups dropped
149 Lineups
28 Duplicates
0 Infeasible lineups dropped
149 Lineups
29 Duplicates
0 Infeasible lineups dropped
150 Lineups
29 Duplicates


,PG,SG,SF,PF,C,G,F,UTIL
0,Jalen Brunson,Spencer Dinwiddie,Dillon Brooks,LaMarcus Aldridge,Ivica Zubac,Aaron Holiday,Jimmy Butler,DeMar DeRozan
1,Darius Garland,Kevin Porter,Jimmy Butler,Marquese Chriss,Ivica Zubac,De'Aaron Fox,LaMarcus Aldridge,Donovan Mitchell
2,Jalen Brunson,Gary Harris,DeMar DeRozan,Bojan Bogdanovic,Ivica Zubac,Donovan Mitchell,LaMarcus Aldridge,Domantas Sabonis
3,Darius Garland,Gary Harris,Dillon Brooks,Domantas Sabonis,Ivica Zubac,Spencer Dinwiddie,Buddy Hield,Jimmy Butler
4,Collin Sexton,Gary Harris,Kevin Porter,Bojan Bogdanovic,Marquese Chriss,De'Aaron Fox,Jimmy Butler,Zach LaVine
...,...,...,...,...,...,...,...,...
145,Darius Garland,Spencer Dinwiddie,Buddy Hield,Bojan Bogdanovic,Ivica Zubac,De'Aaron Fox,Marquese Chriss,DeMar DeRozan
146,Darius Garland,Jalen Brunson,Jimmy Butler,LaMarcus Aldridge,Ivica Zubac,Collin Sexton,DeMar DeRozan,Domantas Sabonis
147,Jalen Brunson,Gary Harris,Jimmy Butler,Bojan Bogdanovic,Marquese Chriss,Spencer Dinwiddie,LaMarcus Aldridge,Collin Sexton
148,Jalen Brunson,Gary Harris,Buddy Hield,Bojan Bogdanovic,Domantas Sabonis,Collin Sexton,DeMar DeRozan,De'Aaron Fox


In [26]:
players = fast_opt.get_player_distribution(fast_opt.df_lineups)
players[:50]

Ivica Zubac                0.486667
De'Aaron Fox               0.446667
Gary Harris                0.446667
Jimmy Butler               0.426667
Bojan Bogdanovic           0.426667
Spencer Dinwiddie          0.426667
DeMar DeRozan              0.420000
Darius Garland             0.420000
Domantas Sabonis           0.406667
Buddy Hield                0.406667
Donovan Mitchell           0.400000
Jalen Brunson              0.393333
Collin Sexton              0.386667
LaMarcus Aldridge          0.373333
Dillon Brooks              0.333333
Robert Covington           0.293333
Marquese Chriss            0.286667
Kevin Porter               0.246667
Zach LaVine                0.240000
Aaron Holiday              0.193333
Harrison Barnes            0.146667
Richaun Holmes             0.140000
Cedi Osman                 0.133333
Shai Gilgeous-Alexander    0.073333
Kyle Lowry                 0.020000
Bismack Biyombo            0.013333
Alec Burks                 0.006667
Jeremy Lamb                0

## Next, instantiate the generator:

In [15]:
gen = LineupGenerator(df_players, n_lineups_to_optimize=150, batch_size=20,
var_multiple=0.5, drop_fraction=0.4, time_limit=2, verbose=False)

## Now run the generator by giving a target number of generated lineups:

In [16]:
gen.generate(10000)
# gen.df_lineups = pd.read_csv('../data/raw/test_lineups.csv', index_col=0)

2 Infeasible lineups dropped
18 Lineups
0 Duplicates
3 Infeasible lineups dropped
34 Lineups
1 Duplicates
2 Infeasible lineups dropped
52 Lineups
1 Duplicates
1 Infeasible lineups dropped
71 Lineups
1 Duplicates
2 Infeasible lineups dropped
88 Lineups
2 Duplicates
2 Infeasible lineups dropped
104 Lineups
4 Duplicates
1 Infeasible lineups dropped
121 Lineups
5 Duplicates
2 Infeasible lineups dropped
139 Lineups
5 Duplicates
0 Infeasible lineups dropped
148 Lineups
7 Duplicates
1 Infeasible lineups dropped
149 Lineups
7 Duplicates
0 Infeasible lineups dropped
149 Lineups
8 Duplicates
0 Infeasible lineups dropped
149 Lineups
9 Duplicates
0 Infeasible lineups dropped
149 Lineups
10 Duplicates
0 Infeasible lineups dropped
149 Lineups
11 Duplicates
0 Infeasible lineups dropped
150 Lineups
11 Duplicates


,PG,SG,SF,PF,C,G,F,UTIL
0,Darius Garland,Jalen Brunson,DeMar DeRozan,Cedi Osman,LaMarcus Aldridge,Spencer Dinwiddie,Domantas Sabonis,Collin Sexton
1,Spencer Dinwiddie,Gary Harris,Dillon Brooks,Bojan Bogdanovic,Marquese Chriss,Collin Sexton,Buddy Hield,Donovan Mitchell
2,Darius Garland,Spencer Dinwiddie,Buddy Hield,Bojan Bogdanovic,Ivica Zubac,De'Aaron Fox,Marquese Chriss,DeMar DeRozan
3,Darius Garland,Jalen Brunson,Jimmy Butler,Marquese Chriss,Ivica Zubac,Donovan Mitchell,DeMar DeRozan,Domantas Sabonis
4,Collin Sexton,Gary Harris,Buddy Hield,Bojan Bogdanovic,Ivica Zubac,De'Aaron Fox,DeMar DeRozan,LaMarcus Aldridge
...,...,...,...,...,...,...,...,...
145,Spencer Dinwiddie,Gary Harris,Kevin Porter,LaMarcus Aldridge,Ivica Zubac,Jimmy Butler,Dillon Brooks,Zach LaVine
146,Darius Garland,Spencer Dinwiddie,Buddy Hield,Bojan Bogdanovic,Marquese Chriss,Collin Sexton,Harrison Barnes,Donovan Mitchell
147,Jalen Brunson,Gary Harris,DeMar DeRozan,Bojan Bogdanovic,Marquese Chriss,De'Aaron Fox,LaMarcus Aldridge,Donovan Mitchell
148,Spencer Dinwiddie,Collin Sexton,Dillon Brooks,Bojan Bogdanovic,Ivica Zubac,Donovan Mitchell,Marquese Chriss,Richaun Holmes


## Now visualize the player distribution:

In [17]:
players = gen.get_player_distribution(gen.df_lineups)
players[:50]

Ivica Zubac                0.500000
Bojan Bogdanovic           0.486667
Collin Sexton              0.453333
Jimmy Butler               0.453333
Spencer Dinwiddie          0.440000
Gary Harris                0.440000
De'Aaron Fox               0.433333
Buddy Hield                0.433333
Darius Garland             0.426667
Jalen Brunson              0.420000
Donovan Mitchell           0.413333
LaMarcus Aldridge          0.406667
Marquese Chriss            0.400000
DeMar DeRozan              0.393333
Domantas Sabonis           0.353333
Dillon Brooks              0.286667
Zach LaVine                0.266667
Robert Covington           0.220000
Kevin Porter               0.200000
Harrison Barnes            0.186667
Aaron Holiday              0.146667
Richaun Holmes             0.100000
Cedi Osman                 0.086667
Shai Gilgeous-Alexander    0.020000
Alec Burks                 0.006667
T.J. Warren                0.006667
Timothe Luwawu-Cabarrot    0.006667
Tim Frazier                0

## Now run the Monte Carlo Simulation

In [19]:
gen.get_lineup_score_dists()

,PG,SG,SF,PF,C,G,F,UTIL,mean,std
0,Collin Sexton,Gary Harris,Kevin Porter,Harrison Barnes,Richaun Holmes,Aaron Holiday,Paul George,Kyle Lowry,264.54,17.620039
1,Jalen Brunson,Gary Harris,Kevin Porter,Kawhi Leonard,Marquese Chriss,Aaron Holiday,Paul George,Kyle Lowry,262.63,15.017997
2,Collin Sexton,Gary Harris,Kevin Porter,Harrison Barnes,Marquese Chriss,Kyle Lowry,Richaun Holmes,Kawhi Leonard,264.81,17.261263
3,Jalen Brunson,Spencer Dinwiddie,Harrison Barnes,Marquese Chriss,Richaun Holmes,Collin Sexton,Paul George,Aaron Holiday,270.06,20.851468
4,Collin Sexton,Gary Harris,Kevin Porter,Harrison Barnes,Richaun Holmes,Shabazz Napier,Paul George,Kyle Lowry,263.37,17.406462
...,...,...,...,...,...,...,...,...,...,...
1995,Darius Garland,Gary Harris,Jimmy Butler,Harrison Barnes,Marquese Chriss,Spencer Dinwiddie,Robert Covington,Donovan Mitchell,279.25,20.311315
1996,Spencer Dinwiddie,Gary Harris,Kevin Porter,Marquese Chriss,Bam Adebayo,Tim Frazier,Jimmy Butler,De'Aaron Fox,277.49,19.496710
1997,Darius Garland,Gary Harris,Kevin Porter,Domantas Sabonis,Bismack Biyombo,De'Aaron Fox,Jimmy Butler,Donovan Mitchell,273.53,17.789721
1998,De'Aaron Fox,Kevin Porter,Dillon Brooks,Bojan Bogdanovic,Bam Adebayo,Buddy Hield,Robert Covington,Shai Gilgeous-Alexander,267.74,18.799925


## Now find the optimal set of lineups:

In [20]:
df_optimal = gen.enforce_exposures()

# Now build the sheet:

In [28]:
sheet_name = 'test_DKEntries2.csv'
# entries = EntriesHandler('../data/raw/' + sheet_name, gen.df, read_lineups=True)
entries = EntriesHandler('../data/raw/' + sheet_name, fast_opt.df, read_lineups=True)

In [29]:
# entries.add_lineups_to_entries(df_optimal) ## Gen + opt lineups
entries.add_lineups_to_entries(fast_opt.df_lineups) ## Fast opt lineups

## Visualize the final sheet

In [30]:
pd.read_csv('../data/raw/test_DKEntries2_v2.csv')

,Entry ID,Contest Name,Contest ID,Entry Fee,PG,SG,SF,PF,C,G,F,UTIL
0,1809803072,NBA $12K mini-MAX [150 Entry Max],83787211,$0.50,14066582,14065998,14066340,14066042,14066415,14066210,14065989,14066053
1,1809803073,NBA $12K mini-MAX [150 Entry Max],83787211,$0.50,14066387,14066428,14065989,14066260,14066415,14066058,14066042,14066015
2,1809803074,NBA $12K mini-MAX [150 Entry Max],83787211,$0.50,14066582,14066354,14066053,14066137,14066415,14066015,14066042,14065994
3,1809803075,NBA $12K mini-MAX [150 Entry Max],83787211,$0.50,14066387,14066354,14066340,14065994,14066415,14065998,14066088,14065989
4,1809803076,NBA $12K mini-MAX [150 Entry Max],83787211,$0.50,14066177,14066354,14066428,14066137,14066260,14066058,14065989,14066033
...,...,...,...,...,...,...,...,...,...,...,...,...
145,1809803217,NBA $12K mini-MAX [150 Entry Max],83787211,$0.50,14066387,14065998,14066088,14066137,14066415,14066058,14066260,14066053
146,1809803218,NBA $12K mini-MAX [150 Entry Max],83787211,$0.50,14066387,14066582,14065989,14066042,14066415,14066177,14066053,14065994
147,1809803219,NBA $12K mini-MAX [150 Entry Max],83787211,$0.50,14066582,14066354,14065989,14066137,14066260,14065998,14066042,14066177
148,1809803220,NBA $12K mini-MAX [150 Entry Max],83787211,$0.50,14066582,14066354,14066088,14066137,14065994,14066177,14066053,14066058
